Depois colocar todo esse código dentro do primeiro script transformation

In [1]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

In [2]:
images_array = []
labels_array = []
possible_labels = ['sem_acidente', 'acidente_com_moto', 'acidente_sem_moto']
frames_dir = 'transformation-result/frames-1fps/'

In [3]:
total_images = 0
for label in possible_labels:
    label_dir = os.path.join(frames_dir, label)
    total_images += len([filename for filename in os.listdir(label_dir) if filename.endswith('.jpg') or filename.endswith('.png')])

In [4]:
sample_image_path = os.path.join(frames_dir, possible_labels[0], os.listdir(os.path.join(frames_dir, possible_labels[0]))[0])
sample_image = np.asarray(Image.open(sample_image_path))
img_shape = sample_image.shape

In [5]:
X_path = os.path.join(frames_dir, "X.npy")
y_path = os.path.join(frames_dir, "y.npy")

In [6]:
X = np.memmap(X_path, dtype='uint8', mode='w+', shape=(total_images, *img_shape))
y = np.memmap(y_path, dtype='int32', mode='w+', shape=(total_images,))

In [7]:
index = 0
for i, label in enumerate(possible_labels):
    label_dir = os.path.join(frames_dir, label)
    for filename in tqdm(os.listdir(label_dir), desc=f"Processando {label}"):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(label_dir, filename)
            try:
                image = Image.open(image_path).convert("RGB")
                image_array = np.asarray(image)

                if image_array.shape != img_shape:
                    print(f"[Aviso] Imagem {filename} tem shape diferente. Pulando.")
                    continue

                X[index] = image_array
                y[index] = i
                index += 1
            except Exception as e:
                print(f"[Erro] {filename}: {e}")

Processando acidente_sem_moto: 100%|████████| 8918/8918 [01:41<00:00, 88.09it/s]


In [8]:
X.flush()
y.flush()

In [9]:
print(img_shape)

(480, 640, 3)


In [10]:
sample_image.dtype

dtype('uint8')

In [11]:
print(total_images)

15750
